## Acceptability, feasibility, and user experiences of NOLA Gem

_WIP - NOT FOR DISTRIBUTION_

_Vizualization scripts for pilot trial ($N$ = 32) analyses of acceptability, feasibility, and user experience with [NOLA Gem](https://www.researchprotocols.org/2023/1/e47151/authors): a geospatially customizable culturally tailored JITAI for violence-affected people living with HIV._

> `nola_gem_acceptability.ipynb`<br>
> Simone J. Skeen (07-12-2025)

### 1. Prepare
Installs, imports, requisite packages; customizes outputs.
***

In [ ]:
%%capture

!pip install ollama
#!pip install python-docx

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import os
import pandas as pd
#import seaborn as sns
import warnings

#from irrCAC.raw import CAC
#from google.colab import drive
#from langchain_community.llms import Ollama
from matplotlib import cm
from matplotlib.lines import Line2D
from sklearn.metrics import cohen_kappa_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.mode.copy_on_write = True

pd.set_option(
    'display.max_columns',
    None,
    )

pd.set_option(
    'display.max_rows',
    None,
    )

#warnings.simplefilter(
#    action = 'ignore',
#    category = FutureWarning,
#    )

for c in (FutureWarning, UserWarning):
    warnings.simplefilter(
        action = 'ignore',
        category = c,
        )

In [ ]:
#fm.fontManager.ttflist += fm.createFontList(['Arial.ttf'])

fm.fontManager.addfont('/content/drive/MyDrive/Colab/Arial.ttf')
plt.rcParams['font.family'] = 'Arial'

**Ollama**<br>
http://localhost:11434/

#### Google Colab

In [ ]:
# mount gdrive (cloud)

drive.mount(
    '/content/drive',
    force_remount = True,
    )

#### Jupyter Lab

In [ ]:
# set wd (local)

wd = 'C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/nola_gem/dissem/skeen,etal_acceptability/code'
os.chdir(wd)
%pwd

### 2. Write
Defines `qualitative.py` module.
***

In [ ]:
%%writefile qualitative.py

import requests
import json
import pandas as pd
import time

def code_texts_deductively_ollama(df, alias, text_column, endpoint_url, prompt_template, model_name):
    '''
    Classifies each row of 'text' column in provided df in accord with human-specified prompt,
    includes chain-of-thought reasoning, returning explanations for classification decision.

    Parameters:
    -----------
    df : pd.DataFrame
        df containing the text to classify.
    alias : str
        alias (for brevity) of the qualitative code to be applied.
    text_column : str
        column name in df containing the text to be analyzed.
    endpoint_url : str
        URL where locally hosted LLM runs.
    prompt_template : str
        prompt text with a placeholder (e.g. '{text}') where the row's text will be inserted.
    model_name : str
        model tasked with qualitative deductive coding.

    Returns:
    --------
   df : pd.DataFrame
        The original df with two new columns per deductive code: '{alias}_llm' (either "0" or "1")
        and '{alias}_expl' (the cahin-of-thought explanation)
    '''

    label_column = f'{alias}_llm'
    explanation_column = f'{alias}_expl'

    # insert cols - necessary for .update() downstream
    
    df[label_column] = None
    df[explanation_column] = None    
    
    results = []

    for idx, row in df.iterrows():
        row_text = row[text_column]

        unique_id = f"[Row ID: {idx}]"
        prompt = f"{unique_id}\n\n" + prompt_template.format(text = row_text)

        response = requests.post(
            endpoint_url,
            headers = {'Content-Type': 'application/json'},
            json = {
                'model': model_name,
                'prompt': prompt,
                'stream': False
            })

        print(f"\n--- Index {idx} ---")
        print("Prompt:")
        print(prompt)
        print(f"Status: {response.status_code}")
        print("Raw response:")
        print(response.text)

        label = None
        explanation = None

        if response.status_code == 200:
            try:
                result_json = response.json()
                raw_response_str = result_json.get('response', ' ')

                cleaned_str = raw_response_str.strip().replace("```json", " ").replace("```", " ").strip()
                parsed_output = json.loads(cleaned_str)

                label = parsed_output.get(label_column)
                explanation = parsed_output.get(explanation_column)

            except (json.JSONDecodeError, KeyError, TypeError) as e:
                print("JSON error:", e)
                print("Bad string:")
                print(cleaned_str)

        # save results by row index for correct matching
        
        results.append({
            'idx': idx,
            label_column: label,
            explanation_column: explanation
            })

        # impose delay - avoid model caching errors
        
        time.sleep(0.25)        
        
    # create result df - align to input df by row index
    
    result_df = pd.DataFrame(results).set_index('idx')
    df.update(result_df)

    return df

#### Import

In [ ]:
%pwd
from qualitative import(
    code_texts_deductively_ollama,
    )

### 2. Describe
Aggregates, tabulates, paradata.
***

#### Daily diary skills recommendations

In [ ]:
wd = 'C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/nola_gem/dissem/skeen,etal_acceptability/data/paradata'
os.chdir(wd)
%pwd

d_diary = pd.read_csv(
    'diary_response.csv',
#    index_col = [0],
    )

valid_tx_ids = {
    'g1008', 'g1011', 'g1014', 'g1019', 'g1023', 'g1025', 'g1027',
    'g1030', 'g1033', 'g1034', 'g1035', 'g1037', 'g1038', 'g1039',
    'g1040', 'g1042', 'g1044', 'g1046', 'g1047', 'g1049', 'g1050',
    'g1051', 'gg1045'
    }

# reset idx

d = d_diary[d_diary['username'].isin(valid_tx_ids)]
d.reset_index(inplace = True, drop = True)


# convert datetime

d['created_at'] = pd.to_datetime(
    d['created_at'], utc = True
    ).dt.tz_convert(None).dt.strftime('%Y-%m-%d %H:%M:%S')

# inspect

d.shape
d.info()
d.head(4)

In [ ]:
#print(d.columns.tolist())
#d.dtypes

In [ ]:
# recommendation_skill_selection - inspect

d['recommendation_skill_selection'] = d['recommendation_skill_selection'].fillna('none')
print(d['recommendation_skill_selection'].unique())

In [ ]:
# get n 

n = d['recommendation_skill_selection'].value_counts(dropna = False)
#pct = d['recommendation_skill_selection'].value_counts(
#    normalize = True, 
#    dropna = False,
#    ) * 100

# merge, display

summary = pd.DataFrame({
    'n': n,
#    'pct': pct.round(2) ### round to 2 decimal places
    })
print(summary)

In [ ]:
# get %: n / N skills recommendations 

10 / 144

#### Completed skills

In [ ]:
d = pd.read_csv(
    'UserSummary.csv',
#    index_col = [0],
    )

# inspect

d.shape
d.info()
d.head(23)

In [ ]:
# * spent_minutes - mdn / iqr

skills_min = [
    'Breathing Retraining spent_minutes', 
    'Journal Writing spent_minutes', 
    'Adaptive Coping spent_minutes',
    'Common Irrational Thoughts spent_minutes', 
    'Choosing a Coping Strategy spent_minutes', 
    'Set a SMART Goal spent_minutes',
    'Meditation spent_minutes',
    ]

# compute mdn / iqr

summary = {}

for skill in skills_min:
    non_zero = d[d[skill] > 0][skill] ### parse for >0 values
    zero_count = (d[skill] == 0).sum() ### count 0 values
    mdn = non_zero.median()
    q1 = non_zero.quantile(0.25)
    q3 = non_zero.quantile(0.75)
    val_range = non_zero.max() - non_zero.min() ### range of >0 values
    
    summary[skill] = {
        'mdn': mdn,
#        'q1': q1,
#        'q3': q3,
        'zero_count': zero_count,
        'non_zero_range': val_range,       
        }

# tabulate, display

summary = pd.DataFrame(summary).T ### transpose for interpretability
print(summary)

In [ ]:
# * completion_count - mdn / iqr

skills_complete = [
    'Breathing Retraining completion_count', 
    'Journal Writing completion_count',
    'Adaptive Coping completion_count',
    'Common Irrational Thoughts completion_count',
    'Choosing a Coping Strategy completion_count',
    'Set a SMART Goal completion_count',
    'Meditation completion_count',
    ]

# compute m (sd)

summary = {}

for skill in skills_complete:
    non_zero = d[d[skill] > 0][skill] ### parse for >0 values
    zero_count = (d[skill] == 0).sum() ### count 0 values
#    mean_val = d[col].mean()
#    std_val = d[col].std()
    mean_val = non_zero.mean()
    std_val = non_zero.std()
    summary[skill] = {
        'mean': mean_val,
        'std': std_val,
        'zero_count': zero_count,
        }

# tabulate, display

summary = pd.DataFrame(summary).T
print(summary)

In [ ]:
# get n

sums = {}

for skill in skills_complete:
    total = d[skill].sum()
    column_sums[skill] = total

# tabulate, display

d_sums = pd.DataFrame.from_dict(
    sums, 
    orient = 'index', 
    columns = ['sum'],
    )
print(d_sums)


In [ ]:
# get %: n / N skills completed

19 / 205

### 3. Visualize
Plots acceptability, usability, etc. findings.
***

In [ ]:
%pwd

In [ ]:
%cd /content/drive/My Drive/Colab/nola_gem_acceptability/inputs/data

d = pd.read_csv(
    'nola_gem_acceptability_no_loc_tx.csv',
    index_col = [0],
    )

d.info()
d.head(3)

In [ ]:
d[[
    'intervention1_rev',
    'intervention2_rev',
    'intervention3_rev',
    'intervention4_rev',
    'intervention5_rev',
    ]].head(30)

In [ ]:
%cd ../../outputs/figures

##### **Fig. 1.** NOLA Gem acceptability: domain general.

In [ ]:
# aesthetics

sns.set_style(
    style = 'whitegrid',
    rc = None,
    )

int_cols = [
    'intervention1_rev',
    'intervention2_rev',
    'intervention3_rev',
    'intervention4_rev',
    'intervention5_rev',
    ]

# customize x-axis labels

x_positions = range(len(int_cols))
x_labels = [
    'useful',
    'easy to understand',
    'satisfactory',
    'likeable',
    'visually appealing',
    ]

# customize y-axis labels

y_labels = {
    1: 'disagree',
    2: 'somewhat\ndisagree',
    3: 'somewhat\nagree',
    4: 'agree',
    }

# uniform color: ng_blue

ng_blue = '#7eb0d5'

# plot

fig, ax = plt.subplots(figsize = (8, 5))

# loop over int1-int5

for i, col in enumerate(int_cols):
    y_vals = d[col].astype(float) ### coerce numeric
    x_jittered = np.random.normal(
        loc = i,
        scale = 0.16, ### inject jitter
        size = len(y_vals),
        )

    # color = custom_colors[col]

    ax.scatter(
        x_jittered,
        y_vals,
        alpha = 0.6,
        s = 35,
        color = ng_blue,
        #edgecolors = 'k',
        linewidths = 0.5,
        label = col,
        )

    # overlay m (sd) by int*

    mean_val = y_vals.mean()
    std_val = y_vals.std()

    ax.errorbar(
        i,
        mean_val,
        yerr = std_val,
        fmt = 'D',
        color = ng_blue,
        #edgecolors = 'k',
        markersize = 8,
        capsize = 0,
        linewidth = 1,
        zorder = 3, ### m (sd) to front
        )

# format

ax.set_xticks(x_positions)
ax.set_xticklabels(
    x_labels,
    rotation = 45,
    ha = 'right',
    fontsize = 9,
    )

# transpose y-axis labels

# hide left y-axis ticks and labels

ax.tick_params(
    axis = 'y',
    left = False,
    labelleft = False,
    )

# create a twin y-axis on the right

ax_right = ax.secondary_yaxis('right')
ax_right.set_yticks(list(y_labels.keys()))
ax_right.set_yticklabels(list(y_labels.values()))

ax.set_title(
    "I found NOLA Gem to be...",
    fontsize = 10,
    )

ax.grid(False)
plt.tight_layout()

# save

plt.savefig(
    'int1_int5_high_res_scatter.png',
    dpi = 300,
    )

# display

plt.show()

##### **Fig. 2a.** NOLA Gem acceptability: perceived helpfulness.

In [ ]:
# aesthetics

sns.set_style(
    style = 'whitegrid',
    rc = None,
    )

int_cols = [
    'intervention6_rev',
    'intervention7_rev',
    'intervention8_rev',
    'intervention9_rev',
    'intervention10_rev',
    ]

# customize x-axis labels

x_positions = range(len(int_cols))
x_labels = [
    'reducing distress',
    'improving mood',
    'facilitating coping',
    'changing habits',
    'learning new skills',
    ]

# customize y-axis labels

y_labels = {
    1: 'disagree',
    2: 'somewhat\ndisagree',
    3: 'somewhat\nagree',
    4: 'agree',
    }

# uniform color: ng_blue

ng_blue = '#7eb0d5'

# plot

fig, ax = plt.subplots(figsize = (8, 5))

# loop over int1-int5

for i, col in enumerate(int_cols):
    y_vals = d[col].astype(float) ### coerce numeric
    x_jittered = np.random.normal(
        loc = i,
        scale = 0.16, ### inject jitter
        size = len(y_vals),
        )

    # color = custom_colors[col]

    ax.scatter(
        x_jittered,
        y_vals,
        alpha = 0.6,
        s = 35,
        color = ng_blue,
        #edgecolors = 'k',
        linewidths = 0.5,
        label = col,
        )

    # overlay m (sd) by int*

    mean_val = y_vals.mean()
    std_val = y_vals.std()

    ax.errorbar(
        i,
        mean_val,
        yerr = std_val,
        fmt = 'D',
        color = ng_blue,
        #edgecolors = 'k',
        markersize = 8,
        capsize = 0,
        linewidth = 1,
        zorder = 3, ### m (sd) to front
        )

# format

ax.set_xticks(x_positions)
ax.set_xticklabels(
    x_labels,
    rotation = 45,
    ha = 'right',
    fontsize = 9,
    )

# hide left y-axis ticks and labels

ax.tick_params(
    axis = 'y',
    left = False,
    labelleft = False,
    )

# create a twin y-axis on the right

ax_right = ax.secondary_yaxis('right')
ax_right.set_yticks(list(y_labels.keys()))
ax_right.set_yticklabels(list(y_labels.values()))

ax.set_title(
    "($a.$) I found NOLA Gem to be helpful in...",
    fontsize = 10,
    )

ax.grid(False)
plt.tight_layout()

# save

plt.savefig(
    'int6_int10_high_res_scatter.png',
    dpi = 300,
    )

# display

plt.show()

##### **Fig. 2b.** NOLA Gem acceptability: perceived helpfulness.

In [ ]:
# aesthetics

sns.set_style(
    style = 'whitegrid',
    rc = None,
    )

int_cols = [
    'intervention11_rev',
    'intervention12_rev',
    'intervention13_rev',
    'intervention14_rev',
    ]

# customize x-axis labels

x_positions = range(len(int_cols))
x_labels = [
    'educational sessions',
    'skills practice',
    'geofencing alerts',
    'daily diary suggested skills',
    ]

# customize y-axis labels

y_labels = {
    1: 'not very helpful',
    2: 'a little\nhelpful',
    3: 'somewhat\nhelpful',
    4: 'very helpful',
    }

# uniform color: ng_blue

ng_blue = '#7eb0d5'

# plot

fig, ax = plt.subplots(figsize = (8, 5))

# loop over int1-int5

for i, col in enumerate(int_cols):
    y_vals = d[col].astype(float).replace(5, np.nan).dropna() ### coerce numeric, convert '5' to NaN and drop
    x_jittered = np.random.normal(
        loc = i,
        scale = 0.16, ### inject jitter
        size = len(y_vals),
        )

    # color = custom_colors[col]

    ax.scatter(
        x_jittered,
        y_vals,
        alpha = 0.6,
        s = 35,
        color = ng_blue,
        #edgecolors = 'k',
        linewidths = 0.5,
        label = col,
        )

    # overlay m (sd) by int*

    mean_val = y_vals.mean()
    std_val = y_vals.std()

    ax.errorbar(
        i,
        mean_val,
        yerr = std_val,
        fmt = 'D',
        color = ng_blue,
        #edgecolors = 'k',
        markersize = 8,
        capsize = 0,
        linewidth = 1,
        zorder = 3, ### m (sd) to front
        )

# format

ax.set_xticks(x_positions)
ax.set_xticklabels(
    x_labels,
    rotation = 45,
    ha = 'right',
    fontsize = 9,
    )

# hide left y-axis ticks and labels

ax.tick_params(
    axis = 'y',
    left = False,
    labelleft = False,
    )

# create a twin y-axis on the right

ax_right = ax.secondary_yaxis('right')
ax_right.set_yticks(list(y_labels.keys()))
ax_right.set_yticklabels(list(y_labels.values()))

ax.set_title(
    "($b.$) I found the NOLA Gem __________ features to be...$^a$",
    fontsize = 10,
    )

ax.grid(False)
plt.tight_layout()

# save

plt.savefig(
    'int11_int14_high_res_scatter.png',
    dpi = 300,
    )

# display

plt.show()

### 4. Code
Applies on-device 8B Llama-human synergistic deductive coding
***

#### Transform qual-ready df

In [ ]:
wd = 'C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/nola_gem/dissem/skeen,etal_acceptability/data'
os.chdir(wd)
%pwd

d = pd.read_csv(
    'nola_gem_acceptability_no_loc_tx.csv',
    index_col = [0],
    )

d.info()
d.head(3)

In [ ]:
#cols = d.columns.tolist()
#print(cols)

In [ ]:
d_qual = d[[
    'open1',
    'open2',
    'open3',
    'open4',
    ]].copy()

In [ ]:
# insert qual code / rationale cells

d_qual[[
    'fctn_sjs', 'fctn_rtnl_sjs', 'lgth_sjs', 'lgth_rtnl_sjs',
    'tmng_sjs', 'tmng_rtnl_sjs', 'attn_sjs', 'attn_rtnl_sjs',
    'gltc_sjs', 'gltc_rtnl_sjs', 'prfc_sjs', 'prfc_rtnl_sjs',
    'ftrs_sjs', 'ftrs_rtnl_sjs', 'strs_sjs', 'strs_rtnl_sjs',
    ]] = ' '

d_qual.info()
d_qual.head(3)

In [ ]:
# transform: long qual format for human/LLM coding

# list qual columns

qual_cols = [
    'open1',
    'open2',
    'open3',
    'open4',
    ]

# dedicate provisional df per qual column

for col in qual_cols:
    keep_cols = [c for c in d_qual.columns if c not in qual_cols or c == col]
    globals()[f'd_qual_{col}'] = d_qual[keep_cols].copy()


In [ ]:
# specify 'item' col for long df

d_qual_open1['item'] = 'open1'
d_qual_open2['item'] = 'open2'
d_qual_open3['item'] = 'open3'
d_qual_open4['item'] = 'open4'

# rename all qual cols: 'text'

d_qual_open1 = d_qual_open1.rename(
    columns = {'open1': 'text'},
    )
d_qual_open2 = d_qual_open2.rename(
    columns = {'open2': 'text'},
    )
d_qual_open3 = d_qual_open3.rename(
    columns = {'open3': 'text'},
    )
d_qual_open4 = d_qual_open4.rename(
    columns = {'open4': 'text'},
    )

# concatenate: long df

d_qual_long = pd.concat(
    [d_qual_open1, d_qual_open2,
     d_qual_open3, d_qual_open4],
    axis = 0,
    ignore_index = False,
    )

# reorder

d_qual_long = d_qual_long.reindex(columns = [
    'item', 'text',
    'fctn_sjs',	'fctn_rtnl_sjs', 'lgth_sjs', 'lgth_rtnl_sjs', 'tmng_sjs', 'tmng_rtnl_sjs',
    'attn_sjs',	'attn_rtnl_sjs', 'gltc_sjs', 'gltc_rtnl_sjs', 'prfc_sjs', 'prfc_rtnl_sjs',
    'ftrs_sjs',	'ftrs_rtnl_sjs', 'strs_sjs', 'strs_rtnl_sjs',
    ])

d_qual_long.info()
d_qual_long.head(3)

In [ ]:
#%pwd
%cd ../../inputs/data

# export

d_qual_long.to_excel('nola_gem_acceptability_qual.xlsx')

#### Formulate deductive coding prompts

In [ ]:
role = '''
    You are tasked with applying pre-defined qualitative codes to segments of text excerpted 
    from interviews with users of a mental health app for people living with HIV. 
    
    The app included text-messaged surveys multiple times a day, as well as self-directed sessions 
    on mindfulness meditation, trauma psychoeducation (etc.), and momentary prompts to engage with 
    brief coping skills training as needed throughout the day. 
    
    You will be provided a definition, instructions, and 
    key exemplars of text to guide your coding decisions.
    '''

text = '''
    Text:
    {text}
    '''

##### _EMA questionnaire length_ (alias: `lgth`)

In [ ]:
code = 'EMA questionnaire length'
alias = 'lgth'
code_def = '''
    Describes frustration with text message survey, daily diary, or questionnaire (all refer to the same measure) length. 
    These frustrations can include aspects such as the number of questions, or the amount of time allotted to complete the 
    questionnaire, or the nature of the questions administered to participants.
    
    Frustrations must be directly attributable to the duration of the survey or the time it takes to respond to, specifically
    and explicitly. Vague dissatisfaction with the survey does not warrant a 'lgth' = 1. 
    '''
code_ex = '''
    - "The questions they texted you, there was just too many…the questions seemed to ask the same thing over and again"
    - "The number of questions was too many. It's inconvenient especially if you have typical work and family responsibilities"
    - "There are so many questions on the survey and they don't give you enough time to finish them all"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

lgth_prompt = f'{role}{definition}{instruction}{text}{example}'
print(lgth_prompt)

##### _EMA prompt timing_ (alias: `tmng`)

In [ ]:
code = 'EMA prompt timing'
alias = 'tmng'
code_def = '''
    Describes frustration with text message survey, daily diary, or questionnaire (all refer to the same measure) timing during the day. 
    These frustrations can include the questionnaires being sent too early, or too late, or being difficult to fit in due to obligations
    such as a full-time job.
    
    Frustrations must be directly attributable to the timing of the survey delivery, specifically
    and explicitly. Vague dissatisfaction with the survey does not warrant a 'tmng' = 1. 
    '''
code_ex = '''
    - "I couldn’t always answer the surveys when they sent them during the day…because I’ve got a full-time job"
    - "The timing was all wrong. The diaries should come in earlier or later in the day so I can answer without disrupting my schedule"
    - "The time periods for the text message surveys was unworkable for me. Timing would be better if you could choose yourself"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

tmng_prompt = f'{role}{definition}{instruction}{text}{example}'
print(tmng_prompt)

##### _EMA item attunement_ (alias: `attn`)

In [ ]:
code = 'EMA item attunement'
alias = 'attn'
code_def = '''
    Recounts opinions that text message survey, daily diary, or questionnaire (all refer to the same measure) were inappropriate to 
    or incongruent with app users’ lived experience, including recommendations for more appropriate questions or question wording.
    
    The text must refer to the mismatch between item wording and the user's life experience, explicitly and specifically, to
    warrant a 'attn' = 1. Mentions of formatting issues such as the survey length or timing, alone, do _not_ warrant a 'attn' = 1
    '''
code_ex = '''
    - "Not all of the questions made sense to me, or really fit into how I thik about my life"
    - "The questions felt too vague. I wasn't sure what they meant"
    - "More questions on prayer and surivival, less about drugs and violence"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

attn_prompt = f'{role}{definition}{instruction}{text}{example}'
print(attn_prompt)

##### “_Gotta get those glitches fixed_ [in vivo]” (alias: `gltc`)

In [ ]:
code = 'Glitch fixes'
alias = 'gltc'
code_def = '''
    Describes any app (including text message delivery) malfunctions.
    '''
code_ex = '''
    - "It just froze while I was answering my survey one day"
    - "I feel like it could be great but it felt too glitchy for now"
    - "The glitches. I just couldn't deal with the glitches"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

gltc_prompt = f'{role}{definition}{instruction}{text}{example}'
print(gltc_prompt)

##### "_This app is perfect_ [in vivo]" (alias: `prfc`)

In [ ]:
code = 'Perfection'
alias = 'prfc'
code_def = '''
    Captures all responses that express satisfaction with the pilot app or reject any suggestions for updated features.
    
    Positive sentiment towards the app, alone, does _not_ warrant a 'prfc' = 1; statements that nothing needs to be changed 
	must be specific and explicit to warrant a 'prfc' = 1.
    '''
code_ex = '''
    - "I don't think you need to put in any changes at all"
    - "There was nothing I didn't like"
    - "I liked it all"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

prfc_prompt = f'{role}{definition}{instruction}{text}{example}'
print(prfc_prompt)

##### Tweaks and fresh features (alias: `ftrs`)

In [ ]:
code = 'Tweaks and fresh features'
alias = 'ftrs'
code_def = '''
    Captures all specific recommendations for alterations to existing app features or expressed wishes for 
    entirely new features to be added to the app.
    
    New features must be suggested clearly and specifically to warrant a 'ftrs' = 1; dissatisfaction with 
    existing features alone does _not_ warrant a 'ftrs' = 1.
    '''
code_ex = '''
    - "More coping skill should be added"
    - "I wish it had less text, more multimedia"
    - "The questions should vbe more personalized use to user to better fit our needs"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

ftrs_prompt = f'{role}{definition}{instruction}{text}{example}'
print(ftrs_prompt)

##### “_You seem a little stressed…it will tell you what you can do_ [in vivo]” (alias: `strs`)

In [ ]:
code = 'JITAI recognition'
alias = 'strs'
code_def = '''
    Captures any expression of appreciation for the app's features being offered when needed throughout the day,
    or tailored to particular stressors reported by the user.
    
    Positive sentiment towards the app, alone, does _not_ warrant a 'strs' = 1; descriptions of the app being uniquely 
    responsive to users' needs must be specific and explicit to warrant a 'strs' = 1.
    '''
code_ex = '''
    - "I liked how different meditations were offered for different stresses you mention throughout the day"
    - "The daily check-ins and quick follow ups to try and help you right when you most need it"
    - "Learning all these new skills to stay calm and collected during the day"
    '''

# ----------------------------------------------------------------------------------------- #
definition = f'''
    Definition of "{code}": {code_def}.
    '''

instruction = f'''
    You will be provided with a piece of text. For each piece of text:
    - If it meets the definition of "{code}," output {alias}_llm as "1".
    - Otherwise, output {alias}_llm as "0".
    - Also provide a short explanation in exactly two sentences, stored in {alias}_expl.

    Please respond in valid JSON with keys "{alias}_llm" and "{alias}_expl" only.
    '''

example = f'''
    Below are human-validated examples of "{code}"

    - "{code_ex}"
    '''
# ----------------------------------------------------------------------------------------- #

# concatenate prompt as f-string

strs_prompt = f'{role}{definition}{instruction}{text}{example}'
print(strs_prompt)

#### Dual-code human-coded qual data

In [ ]:
#%cd /content/drive/My Drive/Colab/nola_gem_acceptability/inputs/data

os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/nola_gem/dissem/skeen,etal_acceptability/data/qual')

d = pd.read_excel(
    'nola_gem_acceptability_qual_sjs.xlsx',
    index_col = [0],
    )

d.info()
d.head(3)

In [ ]:
#%%capture

# locally hosted Ollama endpoint

ollama_endpoint = 'http://localhost:11434/api/generate'

# define aliases, prompts

prompts = [
    ('lgth', lgth_prompt), ### (alias, prompt_template)
    ('tmng', tmng_prompt),
    ('attn', attn_prompt),
    ('gltc', gltc_prompt),
    ('prfc', prfc_prompt),
    ('ftrs', ftrs_prompt),
    ('strs', strs_prompt),
    ]

# loop through each alias, prompt

for alias, prompt_template in prompts:

# apply code_texts_deductively_ollama over aliases, prompts, updated_df

    d_coded = code_texts_deductively_ollama(
        d,
        alias = alias,
        text_column = 'text',
        endpoint_url = ollama_endpoint,
        prompt_template = prompt_template,
        model_name = 'gemma3:12B',
        )

d_coded.head(3)

# export

d_coded.to_excel('d_coded.xlsx')

#### Compute Cohen's $\kappa$

In [ ]:
os.chdir('C:/Users/sskee/OneDrive/Documents/02_tulane/01_research/nola_gem/dissem/skeen,etal_acceptability/data/qual')
%pwd

In [ ]:
d = pd.read_excel(
    'd_coded.xlsx',
    index_col = [0],
    )

encodings_sjs = [
    'lgth_sjs', 
    'tmng_sjs',
    'attn_sjs',
    'gltc_sjs',
    'prfc_sjs',
    'ftrs_sjs',
    'strs_sjs',
    ]

# numeric conversion - coerce

for e in encodings_sjs:
    d[e] = pd.to_numeric(d[e], errors = 'coerce')

# replace NaN w/ 0    
    
d[encodings_sjs] = d[encodings_sjs].fillna(0)

# inspect

d.info()
d.head(3)

In [ ]:
# define kappa fx

def calculate_kappa(d, col1, col2):
    return cohen_kappa_score(d[col1], d[col2])

col_pairs = [
    ('lgth_sjs', 'lgth_llm'), 
    ('tmng_sjs', 'tmng_llm'),
    ('attn_sjs', 'attn_llm'),
    ('gltc_sjs', 'gltc_llm'),
    ('prfc_sjs', 'prfc_llm'),
    ('ftrs_sjs', 'ftrs_llm'),
    ('strs_sjs', 'strs_llm'),
    ]

# initialize dict

kappa_results = {}

# % agreement loop

def calculate_percent_agreement(df, col_pairs):
    results = {}
    for col1, col2 in col_pairs:
        agreement = df[col1] == df[col2]
        percent_agreement = (agreement.sum() / len(df)) * 100
        results[f"{col1} & {col2}"] = percent_agreement
    return results

percent_agreement_results = calculate_percent_agreement(d, col_pairs)

for pair, percent in percent_agreement_results.items():
    print(f"Percent agreement for {pair}: {percent:.2f}%")

print("\n")

# kappa loop

for col1, col2 in col_pairs:
    kappa = calculate_kappa(d, col1, col2)
    kappa_results[f'{col1} and {col2}'] = kappa

for pair, kappa in kappa_results.items():
    print(f"Cohen's Kappa for {pair}: {kappa:.2f}")
    
print("\n")

##### _drop_ `ftrs`

In [ ]:
d = d.drop(columns = [col for col in d.columns if col.startswith(('ftrs_', 'fctn_'))]) ### drop 'fctn_' cols - superordinate theme

#### Flag disagreements

In [ ]:
#print(d.columns.tolist())

In [ ]:
# flag disagreements fx

def encode_disagreements(row):
    return 1 if row[0] != row[1] else 0

col_dis = [
    ('lgth_sjs', 'lgth_llm', 'lgth_dis'), 
    ('tmng_sjs', 'tmng_llm', 'tmng_dis'),
    ('attn_sjs', 'attn_llm', 'attn_dis'),
    ('gltc_sjs', 'gltc_llm', 'gltc_dis'),
    ('prfc_sjs', 'prfc_llm', 'prfc_dis'),
    ('strs_sjs', 'strs_llm', 'strs_dis'),
    ]

for col1, col2, dis_col in col_dis:
    d[dis_col] = d[[col1, col2]].apply(
        encode_disagreements,
        axis = 1,
        )

# obfuscate human vs. Gemma encodings    
    
# Example: rename _sjs to _a and _llm to _b

#d.columns = [
#    col.replace('_sjs', '_a').replace('_llm', '_b') if col.endswith(('_sjs', '_llm')) else col
#    for col in d.columns
#    ]

# reorder for ST interpretability 

d = d[[
    'item', 'text', 
    'lgth_sjs', 'lgth_rtnl_sjs', 'lgth_llm', 'lgth_expl','lgth_dis', 
    'tmng_sjs', 'tmng_rtnl_sjs', 'tmng_llm', 'tmng_expl', 'tmng_dis', 
    'attn_sjs', 'attn_rtnl_sjs', 'attn_llm', 'attn_expl', 'attn_dis', 
    'gltc_sjs', 'gltc_rtnl_sjs','gltc_llm', 'gltc_expl', 'gltc_dis', 
    'prfc_sjs', 'prfc_rtnl_sjs','prfc_llm', 'prfc_expl', 'prfc_dis', 
    'strs_sjs', 'strs_rtnl_sjs', 'strs_llm', 'strs_expl', 'strs_dis',
    ]]

# inspect

d.info()
d.head(3)    
        
# export

d.to_excel(f'd_coded_icr.xlsx')

> End of nola_gem_acceptability.ipynb